In [39]:
import kfp
import kfp.dsl as dsl
from kfp import components
import kubernetes as k8s
from kfp import compiler, dsl
from kubernetes.client import V1VolumeMount
import pprint
import numpy as np


@dsl.pipeline(
    name="blueberry detection",
    description="blueberry detection pipeline"
)
def full_pipeline(size: str="2Gi"):
    
    vop = dsl.PipelineVolume(
        pvc="biosens-agrovision-data-volume",
        name="biosens-agrovision-data-volume",
    )
    
    download_component = kfp.components.load_component_from_file('download/download_model.yaml')
    test_component = kfp.components.load_component_from_file('test_component.yaml')
    postprocessing_model = kfp.components.load_component_from_file('postprocessing/postprocessing_model.yaml')
    
    
    
    download_component_task = download_component(
        "biosens-test",
        "/mnt"
        ).add_pvolumes({"/mnt": vop})
    download_component_task.execution_options.caching_strategy.max_cache_staleness = "P0D"
    
    test_component_task = test_component(
        download_component_task.output
        ).add_pvolumes({"/mnt": vop})#.execution_options.caching_strategy.max_cache_staleness = "P0D"
    test_component_task.execution_options.caching_strategy.max_cache_staleness = "P0D"
    
    postprocessing_model = postprocessing_model(
        "/mnt/DataTest/final_results_folder/test_parcel.tif",
        "test-result",
        test_component_task.output
        ).add_pvolumes({"/mnt": vop}) 
    postprocessing_model.execution_options.caching_strategy.max_cache_staleness = "P0D"
    
    
    print("gotovo sve")
if __name__ == '__main__':
    file_name = "full_pipeline.yaml"
    kfp.compiler.Compiler().compile(full_pipeline, file_name)

gotovo sve


In [40]:
import kfp
import random
client=kfp.Client()
filepath = '/home/jovyan/test_working/' + file_name
name = 'blueberry_test_component_' + str(random.randint(30000,90000))
print("Uploaded pipeline:", name)
pipeline = client.pipeline_uploads.upload_pipeline(filepath, name=name)

ERROR:root:Failed to read a token from file '/var/run/secrets/kubeflow/pipelines/token' ([Errno 2] No such file or directory: '/var/run/secrets/kubeflow/pipelines/token').


Uploaded pipeline: blueberry_test_component_62086
